In [1]:
print("okey")

okey


In [ ]:
#from moviepy.editor import VideoFileClip

ModuleNotFoundError: No module named 'moviepy.editor'

In [10]:
import os
import requests

def download_video(url, filename="input_video.mp4"):
    r = requests.get(url)
    with open(filename, "wb") as f:
        f.write(r.content)
    return filename

def extract_audio_with_ffmpeg(video_path, audio_path="extracted_audio.wav"):
    os.system(f"ffmpeg -i {video_path} -q:a 0 -map a {audio_path}")
    return audio_path

In [11]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification
import torch
import torchaudio

# Load model
model_name = "facebook/wav2vec2-large-960h-lv60-self"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)

def predict_accent(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    inputs = processor(waveform.squeeze(), sampling_rate=sample_rate, return_tensors="pt", padding=True)
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_id = torch.argmax(logits, dim=-1).item()
    confidence = torch.softmax(logits, dim=-1).max().item()

    # class names  
    accent_classes = ["American", "British", "Australian"]
    predicted_class = accent_classes[predicted_id % len(accent_classes)]

    return predicted_class, round(confidence * 100, 2)

f:\Agentic RAG\Accent Detection Model usin OpenAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def main(video_url):
    video_path = download_video(video_url)
    audio_path = extract_audio(video_path)
    accent, score = predict_accent(audio_path)
    
    print("Accent Detected:", accent)
    print("Confidence Score:", score, "%")